In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score 
from sklearn.model_selection import  StratifiedShuffleSplit

In [49]:
data = pd.read_csv('./database.csv')
data.shape
data.tail()

,label,area,perimeter,aspectRatio,centroidX,centroidY,equivalentDiameter,extent,meanIntensity,orientation,eccentricity,dicomPatientId,previousCancerDiagnosis
72,0,1006,631.771645,1.545455,140.037773,444.460239,35.789370,0.336230,0.332172,-1.542073,0.815709,3,0.5
73,0,342,150.953319,0.500000,228.985380,441.038012,20.867389,0.527778,0.496996,-0.096871,0.874032,4,1.0
74,0,324,155.539105,0.473684,28.179012,451.783951,20.310825,0.473684,0.539240,0.136800,0.887465,4,1.0
75,1,154,54.870058,1.153846,160.519481,440.194805,14.002817,0.789744,0.379374,1.205650,0.598403,4,1.0
76,0,139,56.870058,0.800000,220.359712,432.712230,13.303394,0.772222,0.488278,-0.220644,0.638744,4,1.0


In [50]:
"""data.centroidX= (data.centroidX - data.centroidX.min()) / (data.centroidX.max() - data.centroidX.min())
data.centroidY= (data.centroidY - data.centroidY.min()) / (data.centroidY.max() - data.centroidY.min())
data.orientation = (data.orientation  - data.orientation.min()) / (data.orientation.max() - data.orientation.min())
data.perimeter= (data.perimeter - data.perimeter.min()) / (data.perimeter.max() - data.perimeter.min())
data.meanIntensity= (data.meanIntensity - data.meanIntensity.min()) / (data.meanIntensity.max() - data.meanIntensity.min())
data.extent= (data.extent - data.extent.min()) / (data.extent.max() - data.extent.min())
data.aspectRatio= (data.aspectRatio - data.aspectRatio.min()) / (data.aspectRatio.max() - data.aspectRatio.min())
data.area = (data.area - data.area.min()) / (data.area.max() - data.area.min())
data.equivalentDiameter = (data.equivalentDiameter - data.equivalentDiameter.min()) / (data.equivalentDiameter.max() - data.equivalentDiameter.min())"""
data = data.drop(columns=['previousCancerDiagnosis', 'dicomPatientId'])
data.head()

,label,area,perimeter,aspectRatio,centroidX,centroidY,equivalentDiameter,extent,meanIntensity,orientation,eccentricity
0,0,334,226.681241,1.636364,134.697605,423.724551,20.621882,0.421717,0.320923,-1.429113,0.811104
1,0,334,226.681241,1.636364,134.697605,423.724551,20.621882,0.421717,0.320923,-1.429113,0.811104
2,0,342,150.953319,0.500000,228.985380,441.038012,20.867389,0.527778,0.496996,-0.096871,0.874032
3,0,138,61.698485,0.800000,36.362319,428.847826,13.255454,0.766667,0.489031,0.155235,0.602152
4,1,108,53.319805,1.000000,122.601852,213.833333,11.726460,0.750000,0.321387,0.929097,0.418157


In [51]:
featuresColumns = [x for x in data.columns if x not in 'label']
stf = StratifiedShuffleSplit(n_splits=1, test_size=2, random_state=42)
trainIdx, testIdx = next(stf.split(data[featuresColumns], data['label']))

xTrain = data.loc[trainIdx, featuresColumns]
yTrain = data.loc[trainIdx, 'label']

xTest = data.loc[testIdx, featuresColumns]
yTest = data.loc[testIdx, 'label']

In [52]:
dt = DecisionTreeClassifier(random_state=2)
dt.fit(xTrain, yTrain)

DecisionTreeClassifier(random_state=2)

In [53]:
dt.tree_.node_count, dt.tree_.max_depth

(9, 4)

In [54]:
def measureErrors(yTrue, yGuess, label):
    return pd.Series({
        'accuracy': accuracy_score(y_true=yTrue, y_pred=yGuess),
        'precision': precision_score(y_true=yTrue, y_pred=yGuess),
        'recall': recall_score(y_true=yTrue, y_pred=yGuess),
        'roc': roc_auc_score(yTrue,yGuess)
        },name=label
    )

In [55]:
trainPrediction = dt.predict(xTrain)
testPrediction = dt.predict(xTest)

trainAndTestErrors = pd.concat([measureErrors(yTrain, trainPrediction, 'train'), measureErrors(yTest, testPrediction, 'test')], axis=1)

trainAndTestErrors

,train,test
accuracy,1.0,0.850000
precision,1.0,0.750000
recall,1.0,0.857143
roc,1.0,0.851648


In [2]:
import pickle;
#pickle.dump(dt, open('dt.sav', 'wb'));

In [3]:

loaded_model = pickle.load(open('dt.sav', 'rb'));


FileNotFoundError: [Errno 2] No such file or directory: 'dt.sav'

In [58]:
predictData = [24,15.6568542495,1.2,128.5,318,5.5279063915000002,0.80000000000000004,0.43202614379999998,1.3211229659999999,0.64881682090000004]
prediction = loaded_model.predict([predictData]);
prob = loaded_model.predict_proba([predictData])
print(prediction[0], prob[0]);

area                  191.000000
perimeter             107.497475
aspectRatio             0.809524
centroidX             150.581152
centroidY             389.151832
equivalentDiameter     15.594510
extent                  0.535014
meanIntensity           0.328796
orientation             0.604204
eccentricity            0.864884
Name: 12, dtype: float64 1
1 [0. 1.]


In [59]:
trainPrediction = loaded_model.predict(xTrain)
testPrediction = loaded_model.predict(xTest)

trainAndTestErrors = pd.concat([measureErrors(yTrain, trainPrediction, 'train'), measureErrors(yTest, testPrediction, 'test')], axis=1)
trainAndTestErrors

,train,test
accuracy,1.0,0.850000
precision,1.0,0.750000
recall,1.0,0.857143
roc,1.0,0.851648
